In [1]:
import pandas as pd
import kagglehub

/home/joshua/Documents/Projects/.venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = kagglehub.dataset_download("ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training")
org_data = pd.read_csv('/home/joshua/.cache/kagglehub/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training/versions/1/dirty_cafe_sales.csv')

In [3]:
clean_data = org_data.drop_duplicates()

In [14]:
clean_data.head(20)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5.0,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,UNKNOWN,3.0,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4.0,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,NaN,5.0,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5.0,4.0,20.0,NaN,In-store,2023-12-31


In [4]:
clean_data['Quantity'] = pd.to_numeric(clean_data['Quantity'], errors='coerce')
clean_data['Price Per Unit'] = pd.to_numeric(clean_data['Price Per Unit'], errors='coerce')
clean_data['Total Spent'] = pd.to_numeric(clean_data['Total Spent'], errors='coerce')

In [8]:
for index, row in clean_data.iterrows(): 
    quantity = row['Quantity']
    price_per_unit = row['Price Per Unit']
    total_spent = row['Total Spent']

    if pd.isna(quantity) and pd.notna(price_per_unit) and pd.notna(total_spent): 
        clean_data.at[index, 'Quantity'] = total_spent / price_per_unit
    elif pd.isna(price_per_unit) and pd.notna(quantity) and pd.notna(total_spent): 
        clean_data.at[index, 'Price Per Unit'] = total_spent / quantity
    elif pd.isna(total_spent) and pd.notna(quantity) and pd.notna(price_per_unit): 
        clean_data.at[index, 'Total Spent'] = quantity * price_per_unit

clean_data['Quantity'] = clean_data['Quantity'].fillna(0)
clean_data['Price Per Unit'] = clean_data['Price Per Unit'].fillna(0)
clean_data['Total Spent'] = clean_data['Total Spent'].fillna(0)

In [7]:
clean_data.loc[clean_data['Price Per Unit']==1.5, 'Item'] = 'Tea'
clean_data.loc[clean_data['Price Per Unit']==2, 'Item'] = 'Coffee'
clean_data.loc[clean_data['Price Per Unit']==1, 'Item'] = 'Cookie'
clean_data.loc[clean_data['Price Per Unit']==5, 'Item'] = 'Salad'

clean_data.loc[clean_data['Item']=='Tea', 'Price Per Unit'] = 1.5
clean_data.loc[clean_data['Item']=='Coffee', 'Price Per Unit'] = 2
clean_data.loc[clean_data['Item']=='Cookie', 'Price Per Unit'] = 1
clean_data.loc[clean_data['Item']=='Salad', 'Price Per Unit'] = 5

In [ ]:
item_price = {
    "Coffee": 2.0,
    "Cookie": 1.0,
    "Salad": 5.0,
    "Tea": 1.5
}

for key, value in item_price.items():
    clean_data.loc[clean_data['Price Per Unit'] == value, 'Item'] = key
    clean_data.loc[clean_data['Item'] == key, 'Price Per Unit'] = value


Coffee
2.0
Cookie
1.0
Salad
5.0
Tea
1.5


In [11]:
clean_data['Item'].replace(['UNKNOWN', 'ERROR'], pd.NA, inplace=True)

/tmp/ipykernel_5715/4100417788.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  clean_data['Item'].replace(['UNKNOWN', 'ERROR'], pd.NA, inplace=True)


In [12]:
o1 = (org_data['Item'] == 'UNKNOWN').sum()
o2 = (org_data['Item'] == 'ERROR').sum()
o3 = org_data['Item'].isna().sum()

c1 = clean_data['Item'].isna().sum()

print(f"Original Item NA :{(o1+o2+o3)}")
print(f"Clean Item NA :{c1}")

Original Item NA :969
Clean Item NA :480
